In [1]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [2]:
# (1)토큰화 를 진행하도록 돕는 클래스 AutoTokenizer

from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

#사용할 체크포인트를 from_pretrained() 함수로 전달 시 매개변수로 전달된 체크포인트의 pretrained 당시의 토큰화 문맥에 맞춰 토큰화 진행
#반환은 딕셔너리 형태로 되며, 이 딕셔너리 안에 저장된 데이터를 바로 모델에 투입 가능

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [3]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

# tokenizer에 문장을 전달하게 되면 두 가지 키를 가지는 딕셔너리 반환
# input_ids 키의 value 값은 문장의 각 단어를 정수화 한 리스트
# attention_mask 키의 value 값은 문장의 단어 부분과 패딩 부분을 구분한 0과 1의 리스트

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [4]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [5]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


In [8]:
# 입력 문장에 대해서 positive 와 negative을 구분하는 분류가 필요
# Automodel이 아니라 AutoModelForSequenceClassification 활용

from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

#output의 모양에 대해 살펴본다면 더 작은 차원으로 축소되었음을 알 수 있음
#우리는 단 2개의 label (positive, negative) 가 있기 때문에 모델의 모양은 2*2가 된다
print(outputs.logits.shape)

torch.Size([2, 2])


In [9]:
# 대부분 아키텍처를 거치게 되면 로짓이 반환
# 즉 이 로짓을 소프트맥스에 넣어서 우리가 원하는 확률값으로 변환하는 과정이 필수적임

print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


In [10]:
import torch

#소프트맥스 함수에 넣어서 원시적인 값 -> 확률값 으로 변환하는 데이터 매핑을 진행
#torch 라이브러리에서 지원하는 nn.functional.softmax 에 매개변수로 로짓값을 전달

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


In [11]:
# 각 위치에 해당하는 라벨을 얻으려면 id2label모델 구성의 속성을 검사

model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}